In [1]:
import datetime
import pandas as pd
import hopsworks
import os
from dotenv import load_dotenv
from functions import utils
import json

import datetime
import requests
import pandas as pd
import hopsworks
import datetime
from pathlib import Path
from functions import utils
import json
import re
import os
import warnings
warnings.filterwarnings("ignore")

In [2]:
load_dotenv()
HOPSWORKS_API_KEY = os.getenv("HOPSWORKS_API_KEY")

os.environ["HOPSWORKS_API_KEY"] = HOPSWORKS_API_KEY
print(HOPSWORKS_API_KEY)

AhdA5UndBaVILBvJ.cvcBxXmN8mo8HPjR1Ryb7XncvNQ6enGsnTHL3LZIUJJ75mpRD0tTptadPTgx4zRu


In [3]:
CITY = "dublin"

## Read the bike data from csv-files

In [4]:
# get directory path and all files in it
df_bike = pd.DataFrame()
for file in os.listdir(CITY):
    if file.endswith(".csv"):
        df_tmp = pd.read_csv(f"{CITY}/" + file,  parse_dates=['datetime'], skipinitialspace=True)
        df_tmp['station'] = file.split(".")[0]
        df_bike = pd.concat([df_bike, df_tmp])

df_bike = df_bike.set_index(['datetime', 'station']).reset_index(drop=False)

df_bike

,datetime,station,num_bikes_available
0,2025-01-02 11:05:44+00:00,AVONDALE_ROAD,22
1,2025-01-02 10:48:53+00:00,AVONDALE_ROAD,22
2,2025-01-02 09:49:18+00:00,AVONDALE_ROAD,22
3,2025-01-02 08:49:10+00:00,AVONDALE_ROAD,23
4,2025-01-02 07:49:04+00:00,AVONDALE_ROAD,23
...,...,...,...
1045033,2023-08-05 16:54:54+00:00,YORK_STREET_WEST,20
1045034,2023-08-05 17:56:37+02:00,YORK_STREET_WEST,16
1045035,2023-08-05 14:51:22+00:00,YORK_STREET_WEST,19
1045036,2023-08-05 15:54:31+02:00,YORK_STREET_WEST,17


In [5]:
df_bike.dropna(inplace=True)
df_bike


,datetime,station,num_bikes_available
0,2025-01-02 11:05:44+00:00,AVONDALE_ROAD,22
1,2025-01-02 10:48:53+00:00,AVONDALE_ROAD,22
2,2025-01-02 09:49:18+00:00,AVONDALE_ROAD,22
3,2025-01-02 08:49:10+00:00,AVONDALE_ROAD,23
4,2025-01-02 07:49:04+00:00,AVONDALE_ROAD,23
...,...,...,...
1045033,2023-08-05 16:54:54+00:00,YORK_STREET_WEST,20
1045034,2023-08-05 17:56:37+02:00,YORK_STREET_WEST,16
1045035,2023-08-05 14:51:22+00:00,YORK_STREET_WEST,19
1045036,2023-08-05 15:54:31+02:00,YORK_STREET_WEST,17


## Get the weather data from Open Mateo

First, we need to get the earliest date of the bike data so that we can get the weater data from Open Mateo for the same period.

In [6]:
earliest_bike_date = pd.Series.min(df_bike['datetime'])
earliest_bike_date = earliest_bike_date.strftime('%Y-%m-%d')
last_bike_date = pd.Series.max(df_bike['datetime'])
last_bike_date = last_bike_date.strftime('%Y-%m-%d')

earliest_bike_date, last_bike_date

('2023-08-05', '2025-01-02')

Then, we get the weather data from Open Mateo

In [7]:
weather_df = utils.get_historical_weather(CITY, earliest_bike_date, last_bike_date)
weather_df = weather_df.rename(columns={'date': 'datetime'})
weather_df

Coordinates 53.32161331176758°N -6.501922607421875°E
Elevation 11.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
                           date  temperature_2m  apparent_temperature  rain  \
0     2023-08-05 00:00:00+00:00       12.889501             11.347359   0.5   
1     2023-08-05 01:00:00+00:00       12.589500             10.415937   4.7   
2     2023-08-05 02:00:00+00:00       12.339500              9.593327   4.4   
3     2023-08-05 03:00:00+00:00       12.589500              9.265682   2.9   
4     2023-08-05 04:00:00+00:00       12.839500              9.678358   2.6   
...                         ...             ...                   ...   ...   
12403 2025-01-02 19:00:00+00:00             NaN                   NaN   NaN   
12404 2025-01-02 20:00:00+00:00             NaN                   NaN   NaN   
12405 2025-01-02 21:00:00+00:00             NaN                   NaN   NaN   
12406 2025-01-02 22:00:00+00:00             NaN                   NaN   NaN   
12407

AttributeError: 'NoneType' object has no attribute 'rename'

## Data validation rules

First, we validate that the number of bikes avaliable is always greater than or equal to zero.

In [8]:
if 'num_bikes_available' not in df_bike.columns:
    raise ValueError("The column 'num_bikes_available' does not exist in the dataframe.")

invalid_rows = df_bike[(df_bike['num_bikes_available'] < 0) | (df_bike['num_bikes_available'] > 100)]

if not invalid_rows.empty:
    print("Validation failed: Some rows have invalid values in 'num_bikes_available'.")
    print(invalid_rows)
    assert False
else:
    print("Validation passed: All values in 'num_bikes_available' are within the range 0 to 100.")
    assert True

Validation passed: All values in 'num_bikes_available' are within the range 0 to 100.


Then, we validate that the rainfall, snowfall, and wind speed are always greater than or equal to zero, and below 1000.

In [9]:
if 'rain_sum' not in weather_df.columns:
    raise ValueError("The column 'rain_sum' does not exist in the dataframe.")
if 'snowfall_sum' not in weather_df.columns:
    raise ValueError("The column 'snowfall_sum' does not exist in the dataframe.")
if 'wind_speed_10m_max' not in weather_df.columns:
    raise ValueError("The column 'wind_speed_10m_max' does not exist in the dataframe.")

invalid_rows = weather_df[(weather_df['rain_sum'] < 0) | (weather_df['rain_sum'] > 1000) |
                          (weather_df['snowfall_sum'] < 0) | (weather_df['snowfall_sum'] > 1000) |
                          (weather_df['wind_speed_10m_max'] < 0) | (weather_df['wind_speed_10m_max'] > 1000)]

if not invalid_rows.empty:
    print("Validation failed: Some rows have invalid values in the specified columns.")
    print(invalid_rows)
    assert False
else:
    print("Validation passed: All values in the specified columns are within the range 0 to 1000.")
    assert True


Validation passed: All values in the specified columns are within the range 0 to 1000.


## Connect to Hopsworks and create two feature groups (+more)

In [10]:
project = hopsworks.login()
secrets = utils.secrets_api(project.name)

fs = project.get_feature_store() 

Connected. Call `.close()` to terminate connection gracefully.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1207494
Connected. Call `.close()` to terminate connection gracefully.


2025-01-03 15:00:34,581 INFO: Initializing external client
2025-01-03 15:00:34,582 INFO: Base URL: https://c.app.hopsworks.ai:443
Connected. Call `.close()` to terminate connection gracefully.


Save the city location to Hopsworks for easy access later.

In [12]:
latitude, longitude = utils.get_city_coordinates(CITY)

secrets_dict = {
    "city": CITY,
    "latitude": latitude,
    "longitude": longitude,
}

secrets_json = json.dumps(secrets_dict)

try:
    secrets.create_secret("CITY_JSON", secrets_json)
except hopsworks.RestAPIError:
    print("CITY_JSON already exists. To update, delete the secret in the UI (https://c.app.hopsworks.ai/account/secrets) and re-run this cell.")
    existing_key = secrets.get_secret("CITY_JSON").value
    print(f"{existing_key}")

2025-01-03 15:01:01,638 WARNING: Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)")': /search?q=dublin&format=json&limit=1
Secret created successfully, explore it at https://c.app.hopsworks.ai:443/account/secrets


### Feature group 1: Bike data

In [14]:
bike_fg = fs.get_or_create_feature_group(
    name='bike_data',
    description='How many bikes are available at each station at a given time',
    version=1,
    primary_key=["datetime", "station"],
    event_time="datetime"
)

bike_fg.insert(df_bike)

bike_fg.update_feature_description("num_bikes_available", "Number of bikes available at the station")
bike_fg.update_feature_description("datetime", "Timestamp of the observation")
bike_fg.update_feature_description("station", "Name of the station")

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1207494/fs/1195126/fg/1394505


Uploading Dataframe: 0.00% |          | Rows 0/1045038 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: bike_data_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
http://c.app.hopsworks.ai/p/1207494/jobs/named/bike_data_1_offline_fg_materialization/executions


### Feature group 2: Weather data

In [17]:
weather_fg = fs.get_or_create_feature_group(
    name='weather_data',
    description='Weather data for the city',
    version=1,
    primary_key=["datetime"],
    event_time="datetime"
)

weather_fg.insert(weather_df)

# datetime	temperature_2m_mean	apparent_temperature_mean	daylight_duration	rain_sum	snowfall_sum	wind_speed_10m_max	wind_direction_10m_dominant	

weather_fg.update_feature_description("datetime", "Timestamp of the observation")
weather_fg.update_feature_description("temperature_2m_mean", "Temperature in Celsius")
weather_fg.update_feature_description("apparent_temperature_mean", "Apparent temperature in Celsius")
weather_fg.update_feature_description("daylight_duration", "Duration of daylight")
weather_fg.update_feature_description("rain_sum", "Rain percipitation in mm")
weather_fg.update_feature_description("snowfall_sum", "Snowfall in mm")
weather_fg.update_feature_description("wind_speed_10m_max", "Wind speed 10 m above ground level")
weather_fg.update_feature_description("wind_direction_10m_dominant", "Dominant wind direction 10 m above ground level")


Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1207494/fs/1195126/fg/1393488


Uploading Dataframe: 0.00% |          | Rows 0/516 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: weather_data_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
http://c.app.hopsworks.ai/p/1207494/jobs/named/weather_data_1_offline_fg_materialization/executions
